Import modules and read data

In [28]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from google.colab import drive
drive.mount('/gdrive')

root = '/gdrive/My Drive/ml_project'

train = pd.read_csv(root + "/input/train.csv")
test = pd.read_csv(root + "/input/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Train shape :  (1306122, 3)
Test shape :  (375806, 2)


Load GoogleNews embeddings

In [ ]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

news_path = root + '/input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

'''
f = open(root + '/input/embeddings/paragram_300_sl999/paragram_300_sl999.txt', 'r')
print ("\n\n\n")

line = f.readline()
print(line)

f.close()

f = open(root + '/input/embeddings/paragram_300_sl999/paragram_300_sl999.txt', 'w')
print ("\n\n\n")


f.seek(0)
f.write("999 300\n")
f.close()


f = open(root + '/input/embeddings/paragram_300_sl999/paragram_300_sl999.txt', 'r')


line = f.readlines()
print(line)




f.close()


glove_file = datapath('test_glove.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)


paragram_path = root + '/input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
paragram_embeddings = KeyedVectors.load_word2vec_format(paragram_path, binary=False, encoding='utf-8')
'''

Define functions that builds vocab and checks the coverage of vocab.

In [30]:
import operator 
import re
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [31]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [55]:

test = train[int(0.8*len(train)):]
train = train[:int(0.8*len(train))]

train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_numbers(x))
train["question_text"] = train["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
sentences = train["question_text"].progress_apply(lambda x: x.split())
to_remove = ['a','to','of','and']
sentences = [[word for word in sentence if not word in to_remove] for sentence in tqdm(sentences)]
vocab = build_vocab(sentences)

100%|██████████| 1044897/1044897 [00:09<00:00, 109755.99it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 1044897/1044897 [00:10<00:00, 95768.21it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 1044897/1044897 [00:04<00:00, 260678.66it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is tr

In [56]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 211814/211814 [00:00<00:00, 242634.17it/s]

Found embeddings for 63.27% of vocab
Found embeddings for  98.97% of all text


Check the top 20 out of vocab.

In [57]:
oov[:20]

[('bitcoin', 784),
 ('Quorans', 693),
 ('Snapchat', 668),
 ('cryptocurrency', 645),
 ('btech', 506),
 ('Brexit', 406),
 ('behaviour', 377),
 ('blockchain', 374),
 ('cryptocurrencies', 371),
 ('upvotes', 338),
 ('programme', 333),
 ('Redmi', 300),
 ('realise', 295),
 ('defence', 281),
 ('KVPY', 269),
 ('Paytm', 268),
 ('grey', 246),
 ('mtech', 224),
 ('…', 224),
 ('Btech', 214)]

Use MLP (2 hidden layer)


Reference : https://wikidocs.net/49071

In [58]:
import statistics
import torch

word_cnt = []
encoding_book = {}

sorted_vocab = sorted(vocab.items(), key=(lambda x:x[1]), reverse = True)
#print(sorted_vocab[:20])

cnt = 1
for pair in sorted_vocab:
  encoding_book[pair[0]] = cnt
  cnt += 1

for sen in sentences:
  word_cnt.append(len(sen))

avg = sum(word_cnt)/len(word_cnt) # avg = 12
#print(avg, statistics.median(word_cnt))

max_words = 15 # one sentence is encoded to 15 words

input = [] # 1306122 * 15 dimension
for sen in sentences:
  encoded = []
  if len(sen) > 15:
    sen = sen[:15]
  for word in sen:
    try:
      encoded.append(encoding_book[word])
    except:
      encoded.append(0)
  if len(encoded) < 15:
    for i in range(15-len(encoded)):
      encoded.append(0)
  
  input.append(encoded)


input_tensor = torch.tensor(input)


In [43]:
print(len(input))
print(len(input[0]), len(input[2]), len(input[222]))
print(input_tensor.shape)

1306122
15 15 15
torch.Size([1306122, 15])


In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np


In [ ]:
X_train = input_tensor
y_train = torch.tensor(np.array(train['target']))

test["question_text"] = test["question_text"].progress_apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: clean_numbers(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
sentences = test["question_text"].progress_apply(lambda x: x.split())
to_remove = ['a','to','of','and']
sentences = [[word for word in sentence if not word in to_remove] for sentence in tqdm(sentences)]

test_input = [] # 1306122 * 15 dimension
for sen in sentences:
  encoded = []
  if len(sen) > 15:
    sen = sen[:15]
  for word in sen:
    try:
      encoded.append(encoding_book[word])
    except:
      encoded.append(0)
  if len(encoded) < 15:
    for i in range(15-len(encoded)):
      encoded.append(0)
  
  test_input.append(encoded)


test_input_tensor = torch.tensor(test_input)

X_test = test_input_tensor
y_test = torch.tensor(np.array(test['target']))

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)





In [80]:

model = Sequential()
model.add(Dense(256, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) # It is binary classification

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.1)
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)

print(score[1])


Epoch 1/10
7347/7347 [==============================] - 26s 3ms/step - loss: 9.4561 - accuracy: 0.9296 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 2/10
7347/7347 [==============================] - 25s 3ms/step - loss: 0.2362 - accuracy: 0.9382 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 3/10
7347/7347 [==============================] - 25s 3ms/step - loss: 0.2390 - accuracy: 0.9382 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 4/10
7347/7347 [==============================] - 25s 3ms/step - loss: 0.2366 - accuracy: 0.9382 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 5/10
7347/7347 [==============================] - 25s 3ms/step - loss: 0.2343 - accuracy: 0.9382 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 6/10
7347/7347 [==============================] - 25s 3ms/step - loss: 0.2327 - accuracy: 0.9382 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 7/10
7347/7347 [==============================] - 26s 3ms/step - loss: 0.2346 - accuracy: 0.9382 - val_loss: 0.2310 - val_ac

In [81]:
test = pd.read_csv(root + "/input/test.csv")

test["question_text"] = test["question_text"].progress_apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: clean_numbers(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
sentences = test["question_text"].progress_apply(lambda x: x.split())
to_remove = ['a','to','of','and']
sentences = [[word for word in sentence if not word in to_remove] for sentence in tqdm(sentences)]

test_input = [] # 1306122 * 15 dimension
for sen in sentences:
  encoded = []
  if len(sen) > 15:
    sen = sen[:15]
  for word in sen:
    try:
      encoded.append(encoding_book[word])
    except:
      encoded.append(0)
  if len(encoded) < 15:
    for i in range(15-len(encoded)):
      encoded.append(0)
  
  test_input.append(encoded)

test_input = np.array(test_input)
test_output = model(test_input)

100%|██████████| 375806/375806 [00:03<00:00, 111313.00it/s]


In [82]:
print(test_output)

tf.Tensor(
[[0.0623641]
 [0.0623641]
 [0.0623641]
 ...
 [0.0623641]
 [0.0623641]
 [0.0623641]], shape=(375806, 1), dtype=float32)
